# December 24, 2022
https://adventofcode.com/2022/day/24

In [1]:
from collections import defaultdict
from queue import PriorityQueue

In [2]:
mini_text = f'''#.#####
#.....#
#>....#
#.....#
#...v.#
#.....#
#####.#'''.split("\n")

In [3]:
test_text = f'''#E######
#>>.<^<#
#.<..<<#
#>v.><>#
#<^v^^>#
######.#'''.split("\n")

In [4]:
fn = "../data/2022/24.txt"
with open(fn, "r") as file:
    puz_text = file.readlines()

puz_text = [line.strip("\n") for line in puz_text]

In [36]:
class BlizzardMap:
    def __init__(self, text):
        # exclude walls
        self.width = len(text[0]) - 2
        self.height = len(text) - 2

        # get a list of blizzards and their pos/dir for each row and col
        self.rbliz = defaultdict(list)
        self.lbliz = defaultdict(list)
        self.ubliz = defaultdict(list)
        self.dbliz = defaultdict(list)

        for y, line in enumerate(text[1:-1]):
            for x, c in enumerate(line[1:-1]):
                if c == "<":
                    self.lbliz[y].append(x)
                elif c == ">":
                    self.rbliz[y].append(x)
                elif c == "^":
                    self.ubliz[x].append(y)
                elif c == "v":
                    self.dbliz[x].append(y)

    def check(self, x, y, t):
        '''check if space x,y is safe at time t'''
        # Assume entrance/exit tiles are safe from blizzards
        if x == 0 and y == -1:
            return True
        if x == self.width-1 and y == self.height:
            return True

        # Boundary check
        if x < 0 or x >= self.width or y < 0 or y >= self.height:
            return False

        # Left Blizzard?
        check = (x+t) % self.width
        if check in self.lbliz[y]:
            return False

        # Right Blizzard?
        check = (x-t) % self.width
        if check in self.rbliz[y]:
            return False

        # Up Blizzard?
        check = (y+t) % self.height
        if check in self.ubliz[x]:
            return False

        # Down Blizzard?
        check = (y-t) % self.height
        if check in self.dbliz[x]:
            return False

        return True

    def get_moves(self, x, y, t):
        '''get possible moves when at x,y at time t'''
        moves = []
        to_check = [ [x+1,y], [x,y+1], [x,y], [x-1,y], [x,y-1] ]
        for xy in to_check:
            if self.check(xy[0], xy[1], t+1):
                moves.append(xy)
        return moves

    def heuristic(self, x, y, goalx, goaly):
        '''best case is we walk to the exit with no backtracking'''
        return abs(goalx - x) + abs(goaly - y)

class Path:
    def __init__(self, x, y, t):
        self.x = x
        self.y = y
        self.time = t

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y and self.time == other.time
    def __ne__(self, other):
        return not (self == other)
    def __lt__(self, other):
        if self.time == other.time:
            if self.y == other.y:
                return self.x < other.x
            else:
                return self.y < other.y
        else:
            return self.time < other.time

    def __hash__(self):
        return hash(str(self))

    def __str__(self):
        return f"({self.x}, {self.y}, {self.time})"
    def __repr__(self):
        return str(self)

    


In [41]:
def walk_in_winter_wonderland( text, start="start", start_time=0 ):
    map = BlizzardMap(text)

    frontier = PriorityQueue()
    # Assuming start is very left in top wall
    # Assuming exit is very right in bottom wall
    if start == "start":
        frontier.put( (0, Path(0, -1, start_time)) )
        goal = [map.width-1, map.height]
    else:
        frontier.put( (0, Path(map.width-1, map.height, start_time)) )
        goal = [0, -1]
    
    # track which x,y,z triplets we've already looked at
    seen = set()


    while not frontier.empty():
        # get next tile to check
        cur = frontier.get()[1]

        if [cur.x, cur.y] == goal:
            break

        for nbr in map.get_moves( cur.x, cur.y, cur.time ):
            # see which moves are available to us and place them in queue
            # Unlike typical A* we don't check to see if we know this tile... we may have to backtrack.
            # We may queue the same triplet multiple times? We'll see if we need to improve that later
            # Note -- turns out avoiding duplicates also avoided an infinite(?) loop
            p = Path( nbr[0], nbr[1], cur.time+1 )
            if p not in seen:
                prio = map.heuristic( nbr[0], nbr[1], goal[0], goal[1] ) + p.time
                frontier.put( (prio, p) )
                seen.add( p )

    if [cur.x, cur.y] == goal:
        return cur.time
    else:
        return None

### Part 1

In [42]:
walk_in_winter_wonderland(mini_text)

10

In [43]:
walk_in_winter_wonderland(test_text)

18

In [44]:
walk_in_winter_wonderland(puz_text)

308

### Part 2

In [47]:
def there_and_back_again_and_there_again( text ):
    # note return times are already cumulative
    first = walk_in_winter_wonderland(text, start = "start")
    then = walk_in_winter_wonderland(text, start = "end", start_time = first)
    lastly = walk_in_winter_wonderland(text, start = "start", start_time = then)
    
    return lastly, [first, then-first, lastly-then]

In [48]:
there_and_back_again_and_there_again( mini_text )

(30, [10, 10, 10])

In [49]:
there_and_back_again_and_there_again( test_text )

(54, [18, 23, 13])

In [50]:
there_and_back_again_and_there_again( puz_text )

(908, [308, 290, 310])